In [108]:
import numpy as np
import pandas as pd
import datetime
import math

In [109]:
dam = pd.read_csv('dam.csv')
dam

,market,date,hour,price,volume
0,1,01/01/2016,0,40.10,477.5
1,1,01/01/2016,1,40.08,486.6
2,1,01/01/2016,2,40.06,495.6
3,1,01/01/2016,3,40.08,496.6
4,1,01/01/2016,4,40.07,502.9
...,...,...,...,...,...
44587,1,29/01/2021,13,61.97,134.0
44588,1,29/01/2021,23,58.05,59.0
44589,1,30/01/2021,17,42.08,64.0
44590,1,31/01/2021,3,42.19,149.0


In [110]:
dam['date']=pd.to_datetime(dam['date'],format="%d/%m/%Y")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam

,market,date,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,
2016-01-01,1,2016-01-01,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,2016-01-01,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,2016-01-01,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2
2016-01-01,1,2016-01-01,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3
2016-01-01,1,2016-01-01,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-29,1,2021-01-29,13,61.97,134.0,2021,1,29,4,2021.01,weekday_13
2021-01-29,1,2021-01-29,23,58.05,59.0,2021,1,29,4,2021.01,weekday_23
2021-01-30,1,2021-01-30,17,42.08,64.0,2021,1,30,5,2021.01,weekend_17


In [111]:
monthly=dam.groupby(['month_no','year','month','hour', 'hour_ref']).mean().reset_index(['hour','month','year'])
monthly=monthly.drop(['day'],1)
monthly.rename(columns={'price': 'price_mean'}, inplace=True)
monthly['volume_sum']=dam[['month_no','hour_ref','volume']].groupby(['month_no','hour_ref']).sum()
monthly['price_stdv']=dam[['month_no','hour_ref','price']].groupby(['month_no','hour_ref']).std()
monthly['price_max']=dam[['month_no','hour_ref','price']].groupby(['month_no','hour_ref']).max()
monthly['price_min']=dam[['month_no','hour_ref','price']].groupby(['month_no','hour_ref']).min()
monthly['price_coefvar']=monthly['price_stdv']/monthly['price_mean']*100
monthly['price_range']=monthly['price_max'] - monthly['price_min']
monthly['hours']=dam[['month_no','hour_ref']].groupby(['month_no','hour_ref']).size()
monthly['power']=monthly['volume_sum']/monthly['hours']
monthly

year  month  hour  market  price_mean      volume  \
month_no hour_ref                                                        
2016.01  weekday_0   2016      1     0     1.0   48.505238  332.371429   
         weekend_0   2016      1     0     1.0   48.618000  372.530000   
         weekday_1   2016      1     1     1.0   48.434286  336.747619   
         weekend_1   2016      1     1     1.0   48.459000  372.900000   
         weekday_2   2016      1     2     1.0   48.385714  339.400000   
...                   ...    ...   ...     ...         ...         ...   
2021.01  weekend_21  2021      1    21     1.0   49.518000   84.900000   
         weekday_22  2021      1    22     1.0   55.550952  109.528571   
         weekend_22  2021      1    22     1.0   52.735000  115.700000   
         weekday_23  2021      1    23     1.0   50.306667  122.947619   
         weekend_23  2021      1    23     1.0   47.540000  119.840000   

                     day_of_week  volume_sum  price_stdv  price_max  \
month_no hour_ref                                                     
2016.01  weekday_0      2.095238      6979.8    3.363676      52.02   
         weekend_0      5.500000      3725.3    2.717061      52.06   
         weekday_1      2.095238      7071.7    3.315599      51.65   
         weekend_1      5.500000      3729.0    2.739337      52.06   
         weekday_2      2.095238      7127.4    3.287444      51.65   
...                          ...         ...         ...        ...   
2021.01  weekend_21     5.500000       849.0    4.600446      55.57   
         weekday_22     2.095238      2300.1    9.181813      78.36   
         weekend_22     5.500000      1157.0    3.677648      58.01   
         weekday_23     2.095238      2581.9    3.931278      58.05   
         weekend_23     5.500000      1198.4    2.531771      51.72   

                     price_min  price_coefvar  price_range  hours       power  
month_no hour_ref                                                              
2016.01  weekday_0       40.10       6.934665        11.92     21  332.371429  
         weekend_0       43.95       5.588590         8.11     10  372.530000  
         weekday_1       40.08       6.845562        11.57     21  336.747619  
         weekend_1       43.80       5.652896         8.26     10  372.900000  
         weekday_2       40.06       6.794244        11.59     21  339.400000  
...                        ...            ...          ...    ...         ...  
2021.01  weekend_21      42.11       9.290453        13.46     10   84.900000  
         weekday_22      46.03      16.528633        32.33     21  109.528571  
         weekend_22      48.39       6.973828         9.62     10  115.700000  
         weekday_23      44.70       7.814625        13.35     21  122.947619  
         weekend_23      42.19       5.325560         9.53     10  119.840000  

[2928 rows x 15 columns]

In [112]:
dam_filt=dam[(dam.year<2020)]
cal_monthly=dam_filt[['month','hour_ref','price']].groupby(['month','hour_ref']).mean()

cal_monthly.rename(columns={'price': 'price_mean'}, inplace=True)
cal_monthly['hours']=dam_filt[['month','hour_ref']].groupby(['month', 'hour_ref']).size()
cal_monthly['volume_sum']=dam_filt[['month','hour_ref','volume']].groupby(['month','hour_ref']).sum()
cal_monthly['power']=cal_monthly['volume_sum']/cal_monthly['hours']
cal_monthly['volume_mean']=cal_monthly['volume_sum']/4
cal_monthly=cal_monthly.drop(['volume_sum'],1)
cal_monthly

price_mean  hours       power  volume_mean
month hour_ref                                              
1     weekday_0    33.593483     89  263.117978     5854.375
      weekday_1    33.247303     89  262.165169     5833.175
      weekday_10   74.164831     89   86.859551     1932.625
      weekday_11   72.023371     89   90.902247     2022.575
      weekday_12   71.558539     89   93.005618     2069.375
...                      ...    ...         ...          ...
12    weekend_5    32.883158     38  257.263158     2444.000
      weekend_6    34.064737     38  241.797368     2297.075
      weekend_7    46.835526     38  216.147368     2053.400
      weekend_8    52.244211     38  207.623684     1972.425
      weekend_9    53.827105     38  206.878947     1965.350

[576 rows x 4 columns]

In [154]:
flow = pd.read_csv('flow.csv')
flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index(pd.DatetimeIndex(flow['date']))
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek
flow['month_no']=flow['year']+flow['month']/100
flow['hour']=flow['hour']-1
flow['hour_ref'] = flow.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)

flow

,hour,flow,from,to,date,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,
2016-01-01,0,160,ZAMZ,ZAMC,2016-01-01,2016,1,1,4,2016.01,weekday_0
2016-01-01,0,170,ZIM,ZAMZ,2016-01-01,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,170,ZIM,ZAMZ,2016-01-01,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,160,ZAMZ,ZAMC,2016-01-01,2016,1,1,4,2016.01,weekday_1
2016-01-01,2,160,ZAMZ,ZAMC,2016-01-01,2016,1,1,4,2016.01,weekday_2
...,...,...,...,...,...,...,...,...,...,...,...
2018-03-24,23,37,ZIM,ZAMZ,2018-03-24,2018,3,24,5,2018.03,weekend_23
2018-03-24,23,37,ZIMA,ZIM,2018-03-24,2018,3,24,5,2018.03,weekend_23
2018-03-24,23,20,LES,RSAN,2018-03-24,2018,3,24,5,2018.03,weekend_23


In [155]:
f_to=flow[['hour','flow','to','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'to': 'node'})
f_from=flow[['hour','flow','from','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']

flows=pd.concat([f_to, f_from])

flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week','month_no','hour_ref']).sum().reset_index()
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,flow
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,160
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,10
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,-170
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,160
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,10
...,...,...,...,...,...,...,...,...,...,...
144343,2018-03-24,23,RSAN,2018,3,24,5,2018.03,weekend_23,24
144344,2018-03-24,23,ZAMC,2018,3,24,5,2018.03,weekend_23,37
144345,2018-03-24,23,ZAMZ,2018,3,24,5,2018.03,weekend_23,0
144346,2018-03-24,23,ZIM,2018,3,24,5,2018.03,weekend_23,0


In [156]:
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,flow
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,160
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,10
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,-170
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,160
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,10
...,...,...,...,...,...,...,...,...,...,...
144338,2018-03-24,22,ZAMZ,2018,3,24,5,2018.03,weekend_22,-25
144341,2018-03-24,23,BOT,2018,3,24,5,2018.03,weekend_23,-41
144342,2018-03-24,23,LES,2018,3,24,5,2018.03,weekend_23,-20
144343,2018-03-24,23,RSAN,2018,3,24,5,2018.03,weekend_23,24


In [157]:
flow_monthly=flows.groupby(['month_no','year','month','hour', 'hour_ref','node']).mean().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_mean'}, inplace=True)
flow_monthly=flow_monthly.drop(['day_of_week','day'],axis=1)
flow_monthly

year  month  hour  flow_mean
month_no hour_ref   node                                  
2016.01  weekday_0  BOT       2016      1     0 -17.750000
                    MOZN_HCB  2016      1     0 -93.214286
                    MOZS      2016      1     0 -14.000000
                    NAM       2016      1     0  25.000000
                    RSAN      2016      1     0 -38.000000
...                            ...    ...   ...        ...
2018.03  weekend_23 BOT       2018      3    23 -50.000000
                    LES       2018      3    23 -17.600000
                    RSAN      2018      3    23  48.142857
                    ZAMC      2018      3    23  26.571429
                    ZAMZ      2018      3    23 -28.333333

[8534 rows x 4 columns]

In [158]:
flow_monthly.to_csv('dam_flow_monthly.csv')

In [44]:
monthly.to_csv('dam_monthly_hours.csv')

In [45]:
cal_monthly.to_csv('dam_cal_monthly_hours.csv')

In [46]:
dam.to_csv('dam_out.csv')